# LangChain: Agents

## Outline:

* Using built in LangChain tools: Google search and Wikipedia
* Defining your own tools

## Built-in LangChain tools

In [1]:
#!pip install -U wikipedia
%pip install -U langchain

In [6]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

In [7]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [9]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [12]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True
)

In [14]:
agent.invoke("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We can use a calculator to find 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```
Observation: Answer: 75.0
Thought:Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

## Wikipedia example

In [15]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent.invoke(question)



> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

/home/fulim/projects/langchain-vector_db/langchain-vector_db/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/fulim/projects/langchain-vector_db/langchain-vector_db/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtai

## Python Agent

In [28]:
from langchain.agents import load_tools, initialize_agent, Tool, AgentType
python_repl = PythonREPL()

In [29]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [33]:
repl_tool = [
    Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
    )
]

In [34]:
agent = initialize_agent(repl_tool,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=6,
                         handle_parsing_errors=True)

#### View detailed outputs of the chains

In [35]:
import langchain
langchain.debug=True
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can. You have access to the following tools:\n\npython_repl(command: str, timeout: Optional[int] = None) -> str - A Python shell

## Define your own tool

In [ ]:
#!pip install DateTime

In [36]:
from langchain.agents import tool
from datetime import date

In [37]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [38]:
agent= initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [39]:
try:
    result = agent("whats the date today?")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I should use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-07-29
Thought:Could not parse LLM output: I now know the final answer.
Observation: Invalid or incomplete response
Thought:I made a mistake in my previous response. Let me correct it and provide you with the accurate date today. 

Action:
```
{
  "action": "time",
  "action_input": ""
}
``` 

Final Answer: 2024-07-29

> Finished chain.
